Код из статьи: [Как запустить Apache AirFlow в Google Colab с ngrok](https://bigdataschool.ru/blog/how-to-launch-airflow-on-colab-with-ngrok.html)

In [1]:
###############################################ячейка №1 в Google Colab#############################
#Установка Apache Airflow и инициализация базы данных
!pip install apache-airflow
!airflow initdb
#Установка инструмента ngrok для создания безопасного туннеля для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok
#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:

In [4]:
###############################################ячейка №2 в Google Colab#############################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow, а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-10-13T06:50:14.598+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-b51243e9-45f0-4c32-8295-02b56b22f211
[2023-10-13T06:50:14.601+0000] {process.py:99} INFO - t=2023-10-13T06:50:14+0000 lvl=info msg=start pg=/api/tunnels id=9afdf18a320645b0
[2023-10-13T06:50:14.644+0000] {process.py:99} INFO - t=2023-10-13T06:50:14+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8888-b51243e9-45f0-4c32-8295-02b56b22f211 addr=http://localhost:8888 url=https://2c49-35-202-131-127.ngrok-free.app
Адрес Airflow GUI: NgrokTunnel: "https://2c49-35-202-131-127.ngrok-free.app" -> "http://localhost:8888"
[2023-10-13T06:50:14.647+0000] {process.py:99} INFO - t=2023-10-13T06:50:14+0000 lvl=info msg=end pg=/api/tunnels id=9afdf18a320645b0 status=201 dur=43.211922ms


In [5]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow
!airflow upgradedb #Обновление базы данных Airflow
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna.doe@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
[2023-10-13T06:52:16.545+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-10-13T06:52:16.546+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 405de8318b3a
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done
Usage: airflow
       [-h]
       GROUP_OR_COMMAND
       ...

Positional Arguments:
  GROUP_OR_COMMAND
    
    Groups
      config
    View
    configurati
    on
      connections
    Manage
    connections
      dags
    Manage DAGs
      db
    Database
    operations
      jobs
    Manage jobs
      pools
    Manage
    pools
      providers
    Display
    providers
      roles
    Manage
    roles
      tasks
    Manage
    tasks
      users
    Manage
    user

In [ ]:
##########################ячейка №5 в Google Colab#######################
#запуск веб-сервера Apache Airflow на порту 8888. Веб-сервер Airflow предоставляет пользовательский интерфейс
#для управления DAG’ами,
#просмотра логов выполнения задач, мониторинга прогресса выполнения
!airflow webserver --port 8888

[2023-10-13T06:53:15.419+0000] {configuration.py:2067} INFO - Creating new FAB webserver config file in: /root/airflow/webserver_config.py
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8888
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:336 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2023-10-13T06:53:18.198+0000] {manager.py:499} INFO - Created Permission View: %s
[2023-10-13T06:53:18.212+0000] {manager.py:555} INFO - Added Permission %s to